In [1]:
!pip install dbfread
!pip install haversine
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dbfread import DBF
from haversine import haversine
from tqdm.notebook import tqdm
%matplotlib inline
import platform
import pickle
import re
platform.system()
if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')
plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
%config InlineBackend.figure_format = 'retina'


In [2]:
def pklopen(route):
    with open('{}'.format(route), 'rb') as f:
        df = pickle.load(f)
    return df

In [3]:
#데이터 연결(버스+지하철)
busT_up_1 = pklopen('./busT_up_1.pkl')
busT_down_1 = pklopen('./busT_down_1.pkl')
busT_up_2 = pklopen('./busT_up_2.pkl')
busT_down_2 = pklopen('./busT_down_2.pkl')
busT_up_3 = pklopen('./busT_up_3.pkl')
busT_down_3 = pklopen('./busT_down_3.pkl')
busT_up_4 = pklopen('./busT_up_4.pkl')
busT_down_4 = pklopen('./busT_down_4.pkl')
busT_up_5 = pklopen('./busT_up_5.pkl')
busT_down_5 = pklopen('./busT_down_5.pkl')

In [4]:
#데이터 연결(시간)
time1902= pd.read_csv(r'./4개지역_FLOW_TIME_201902.CSV', sep = '|')
time1903= pd.read_csv(r'./4개지역_FLOW_TIME_201903.CSV', sep = '|')
time1904= pd.read_csv(r'./4개지역_FLOW_TIME_201904.CSV', sep = '|')
time1905= pd.read_csv(r'./4개지역_FLOW_TIME_201905.CSV', sep = '|')
time2002= pd.read_csv(r'./4개지역_FLOW_TIME_202002.CSV', sep = '|')
time2003= pd.read_csv(r'./4개지역_FLOW_TIME_202003.CSV', sep = '|')
time2004= pd.read_csv(r'./4개지역_FLOW_TIME_202004.CSV', sep = '|')
time2005= pd.read_csv(r'./4개지역_FLOW_TIME_202005.CSV', sep = '|')

In [5]:
group_up = pklopen('./train_group_up.pkl')
group_down = pklopen('./train_group_up.pkl')

In [6]:
sta_dong = pklopen('./지하철역 동 매칭 수정.pkl')

In [7]:
junggu = ['서대문', '충정로', '시청', '회현', '서울역', '을지로입구', '명동', '을지로3가', '을지로4가', '충무로', '동대입구', '약수', '버티고개', '청구', '신당', '동대문역사문화공원', ]
nowon = ['수락산', '마들', '노원', '중계', '하계', '월계', '광운대', '석계', '태릉입구', '화랑대', '공릉', '상계', '당고개']

In [8]:
busT_up_need = pd.concat([busT_up_2, busT_up_3, busT_up_4, busT_up_5])
busT_up_need

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
0,20200201,가락시장,268.574,410.275,677.201,975.433,1036.4,1242.36,1211.05,1454.91,...,1443.38,1552.13,1410.42,1321.45,944.127,1018.27,815.607,757.938,321.3,6.59077
1,20200201,가산디지털단지,1297.82,990.808,1432.72,1800.2,2163.03,1883.93,2144.43,2977.08,...,4991.26,6363.5,8075.32,6996.13,5023.82,3777.17,5009.86,2223.5,976.853,13.955
2,20200201,강남,1101.69,1156.07,1036.43,1463.74,1950.1,2664.87,3820.95,5718.21,...,7944.9,10065.9,10595.8,8919.17,5960.62,5901.57,6908.47,8184.19,4256.03,0
3,20200201,강남구청,148.165,232.505,271.256,428.539,581.262,640.528,711.192,1053.11,...,2092.54,2320.49,2341.01,1937.54,1905.63,1634.37,1568.27,1369.96,480.966,0
4,20200201,강동,425.96,576.622,860.139,1323.08,1443.61,1329.93,1344.99,1531.27,...,1321.71,1447.72,1501.13,1208.03,788.917,701.26,571.143,491.704,182.163,1.36965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,20200531,홍제,611.809,1118.61,1429.08,2282.87,2995.12,2995.12,3008.82,3835.22,...,2721.18,2785.1,2520.29,1945,1479.3,1269.27,995.33,575.283,296.773,9.13147
6878,20200531,화곡,700.907,916.012,1418.73,2397.58,2755.29,2158.31,2271.9,2450.76,...,1832.02,1742.6,1590.33,1280.97,1090.03,889.426,787.916,594.562,219.94,0
6879,20200531,화랑대(서울여대입구),233.314,362.934,518.476,950.54,999.507,1071.52,910.214,1189.62,...,841.084,841.084,797.878,555.922,426.303,348.531,316.847,198.749,112.337,0
6880,20200531,회현(남대문시장),74.7855,132.952,151.648,207.737,261.749,382.237,481.951,814.331,...,1312.9,1146.71,1242.27,1873.79,1005.45,1742.92,531.808,247.207,76.8628,2.07737


In [9]:
busT_down_need = pd.concat([busT_down_2, busT_down_3, busT_down_4, busT_down_5])
busT_down_need

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
0,20200201,가락시장,148.971,487.395,531.115,866.301,1028.23,1054.13,1139.95,1271.11,...,1567.44,1439.52,1771.46,1541.53,1125.38,772.384,735.141,637.986,553.785,202.407
1,20200201,가산디지털단지,458.846,2128.67,2024.6,4701.99,5832.55,4167.46,4437.09,5085.15,...,4711.45,4276.26,3992.43,3434.25,2081.36,1669.82,1381.27,1277.2,1121.1,141.911
2,20200201,강남,514.572,1624.81,2050.92,5061.68,10384.3,7656.66,6994.65,7313.12,...,6685.02,6497.77,7606.53,7006.44,4609.04,3470.78,2679.02,2379.71,2002.26,1.47442
3,20200201,강남구청,81.4128,574.065,880.928,1839.09,3237.72,2277.47,1872.49,1937.21,...,1381.93,1281.73,1070.89,933.116,597.027,478.039,382.014,302.689,248.413,31.3126
4,20200201,강동,57.9538,564.678,533.472,754.885,833.643,907.942,928.746,1121.93,...,1294.3,1527.6,1588.53,1493.42,1144.22,1053.57,1022.36,954.008,918.344,141.169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6877,20200531,홍제,121.061,739.336,683.129,1171.7,1582.44,1846.18,1310.05,1660.26,...,2339.07,2857.9,2836.28,3013.55,2360.69,2256.92,2464.45,2200.71,1249.52,12.9708
6878,20200531,화곡,70.6839,454.396,378.664,535.178,830.535,921.415,891.122,1438.92,...,1908.46,2282.08,2547.14,2269.46,2249.26,2441.12,2425.97,2279.55,1264.74,88.3548
6879,20200531,화랑대(서울여대입구),52.1655,195.621,244.526,498.832,707.494,596.643,661.85,557.519,...,759.66,824.867,909.636,880.292,847.689,824.867,958.541,762.92,635.767,42.3845
6880,20200531,회현(남대문시장),13.522,278.167,436.568,701.212,2022.51,1837.06,1582.07,1444.92,...,867.34,660.646,540.88,367.026,251.123,171.923,282.03,183.513,92.7223,7.72686


In [10]:
with open('버스지하철_총승차.pkl', 'wb') as f:
    pickle.dump(busT_up_need, f)

In [11]:
with open('버스지하철_총하차.pkl', 'wb') as f:
    pickle.dump(busT_down_need, f)

In [12]:
time2002

,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,202002,20200201,1114052000,소공동,1393.34,1144.91,900.51,721.78,784.43,1479.50,...,7705.97,7389.93,7322.32,8070.40,6488.70,4890.46,4304.38,3683.57,4002.98,2894.37
1,202002,20200201,1114054000,회현동,1586.96,1170.81,898.06,682.55,763.84,2089.57,...,11846.92,11701.47,11364.08,10725.66,8140.20,6399.03,5566.32,5533.92,4715.67,3334.92
2,202002,20200201,1114055000,명동,1534.52,1139.65,1015.20,799.85,773.06,1506.43,...,11188.64,10109.60,10587.30,10697.87,8781.11,6420.97,5091.03,5025.40,4415.18,3415.71
3,202002,20200201,1114057000,필동,1321.52,933.49,803.40,690.18,754.61,954.29,...,4931.33,4989.49,5010.95,5702.29,5089.43,3959.67,3151.50,3048.29,2951.06,2017.30
4,202002,20200201,1114058000,장충동,629.17,561.68,454.77,372.22,388.65,627.47,...,3423.15,3030.29,2882.15,3099.08,2584.95,1811.88,1669.96,1664.65,1164.56,976.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,202002,20200229,2726066100,범물1동,485.96,388.68,348.90,289.15,330.33,409.87,...,1809.52,1957.39,2093.72,2040.10,1849.73,1265.70,1041.32,905.75,712.61,628.77
1997,202002,20200229,2726066200,범물2동,947.07,896.23,851.62,794.46,783.83,787.49,...,1270.22,1276.28,1296.90,1351.59,1351.30,1246.78,1140.83,1108.22,1031.76,1024.64
1998,202002,20200229,2726067000,고산1동,2162.60,1758.41,1318.00,1203.27,1040.30,1184.69,...,5968.43,6474.89,6595.47,6772.06,6391.21,5232.65,4209.28,3874.91,3198.69,2708.60
1999,202002,20200229,2726068000,고산2동,2599.19,2085.08,1549.93,1509.08,1406.71,1800.18,...,11057.90,11587.25,12424.47,11691.56,10228.67,7664.57,6384.05,5682.11,4365.86,3399.61


In [13]:
sta_dong

,동,역명
0,가락동,"가락시장, 송파, 경찰병원"
1,가산동,가산디지털단지
2,가양동,"마곡, 발산"
3,갈산동,부평구청
4,갈월동,숙대입구(갈월)
...,...,...
169,홍제동,"홍제, 무악재"
170,화곡동,"까치산, 화곡, 우장산"
171,화양동,"건대입구, 어린이대공원(세종대)"
172,효창동,효창공원앞


## 형식 통일 전처리

In [14]:
del group_up['합 계']
del group_down['합 계']

In [15]:
group_up[group_up['사용일자']==20200131]

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
7263,20200131,가락시장,175,456,1282,1672,1055,871,950,924,...,1086,1185,1513,2796,1338,860,910,764,409,100
7264,20200131,가산디지털단지,301,323,759,975,704,644,750,1082,...,1845,2421,7205,15584,6550,3000,2580,1543,836,123
7265,20200131,강남,438,797,1963,2944,2348,2214,2906,3945,...,6189,6952,9983,15641,10407,7304,8239,9290,5315,1484
7266,20200131,강남구청,64,140,273,375,324,294,370,462,...,972,964,1642,3880,2412,1310,1203,1135,512,111
7267,20200131,강동,448,828,2830,3523,1660,1037,909,932,...,977,1044,1778,2088,1084,615,526,437,214,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7500,20200131,홍제,315,788,2590,3260,1805,1090,949,950,...,920,937,1100,1210,672,430,355,327,159,31
7501,20200131,화곡,852,1605,5173,6855,2927,1599,1370,1386,...,1096,1151,1422,1799,1049,653,594,512,326,48
7502,20200131,화랑대(서울여대입구),193,766,2936,2580,1102,601,449,552,...,526,582,628,592,326,187,155,133,69,10
7503,20200131,회현(남대문시장),78,127,221,384,442,633,921,1144,...,1893,2595,4536,5286,2484,2250,986,653,295,43


In [16]:
group_up = group_up.iloc[7505:]
group_down = group_down.iloc[7505:]
group_down

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
7505,20200201,가락시장,163,249,411,592,629,754,735,883,...,876,942,856,802,573,618,495,460,195,4
7506,20200201,가산디지털단지,279,213,308,387,465,405,461,640,...,1073,1368,1736,1504,1080,812,1077,478,210,3
7507,20200201,강남,709,744,667,942,1255,1715,2459,3680,...,5113,6478,6819,5740,3836,3798,4446,5267,2739,0
7508,20200201,강남구청,65,102,119,188,255,281,312,462,...,918,1018,1027,850,836,717,688,601,211,0
7509,20200201,강동,311,421,628,966,1054,971,982,1118,...,965,1057,1096,882,576,512,417,359,133,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36786,20200531,홍제,134,245,313,500,656,656,659,840,...,596,610,552,426,324,278,218,126,65,2
36787,20200531,화곡,290,379,587,992,1140,893,940,1014,...,758,721,658,530,451,368,326,246,91,0
36788,20200531,화랑대(서울여대입구),81,126,180,330,347,372,316,413,...,292,292,277,193,148,121,110,69,39,0
36789,20200531,회현(남대문시장),36,64,73,100,126,184,232,392,...,632,552,598,902,484,839,256,119,37,1


In [17]:
group_up

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
7505,20200201,가락시장,163,249,411,592,629,754,735,883,...,876,942,856,802,573,618,495,460,195,4
7506,20200201,가산디지털단지,279,213,308,387,465,405,461,640,...,1073,1368,1736,1504,1080,812,1077,478,210,3
7507,20200201,강남,709,744,667,942,1255,1715,2459,3680,...,5113,6478,6819,5740,3836,3798,4446,5267,2739,0
7508,20200201,강남구청,65,102,119,188,255,281,312,462,...,918,1018,1027,850,836,717,688,601,211,0
7509,20200201,강동,311,421,628,966,1054,971,982,1118,...,965,1057,1096,882,576,512,417,359,133,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36786,20200531,홍제,134,245,313,500,656,656,659,840,...,596,610,552,426,324,278,218,126,65,2
36787,20200531,화곡,290,379,587,992,1140,893,940,1014,...,758,721,658,530,451,368,326,246,91,0
36788,20200531,화랑대(서울여대입구),81,126,180,330,347,372,316,413,...,292,292,277,193,148,121,110,69,39,0
36789,20200531,회현(남대문시장),36,64,73,100,126,184,232,392,...,632,552,598,902,484,839,256,119,37,1


In [18]:
time2002

,STD_YM,STD_YMD,HDONG_CD,HDONG_NM,TMST_00,TMST_01,TMST_02,TMST_03,TMST_04,TMST_05,...,TMST_14,TMST_15,TMST_16,TMST_17,TMST_18,TMST_19,TMST_20,TMST_21,TMST_22,TMST_23
0,202002,20200201,1114052000,소공동,1393.34,1144.91,900.51,721.78,784.43,1479.50,...,7705.97,7389.93,7322.32,8070.40,6488.70,4890.46,4304.38,3683.57,4002.98,2894.37
1,202002,20200201,1114054000,회현동,1586.96,1170.81,898.06,682.55,763.84,2089.57,...,11846.92,11701.47,11364.08,10725.66,8140.20,6399.03,5566.32,5533.92,4715.67,3334.92
2,202002,20200201,1114055000,명동,1534.52,1139.65,1015.20,799.85,773.06,1506.43,...,11188.64,10109.60,10587.30,10697.87,8781.11,6420.97,5091.03,5025.40,4415.18,3415.71
3,202002,20200201,1114057000,필동,1321.52,933.49,803.40,690.18,754.61,954.29,...,4931.33,4989.49,5010.95,5702.29,5089.43,3959.67,3151.50,3048.29,2951.06,2017.30
4,202002,20200201,1114058000,장충동,629.17,561.68,454.77,372.22,388.65,627.47,...,3423.15,3030.29,2882.15,3099.08,2584.95,1811.88,1669.96,1664.65,1164.56,976.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,202002,20200229,2726066100,범물1동,485.96,388.68,348.90,289.15,330.33,409.87,...,1809.52,1957.39,2093.72,2040.10,1849.73,1265.70,1041.32,905.75,712.61,628.77
1997,202002,20200229,2726066200,범물2동,947.07,896.23,851.62,794.46,783.83,787.49,...,1270.22,1276.28,1296.90,1351.59,1351.30,1246.78,1140.83,1108.22,1031.76,1024.64
1998,202002,20200229,2726067000,고산1동,2162.60,1758.41,1318.00,1203.27,1040.30,1184.69,...,5968.43,6474.89,6595.47,6772.06,6391.21,5232.65,4209.28,3874.91,3198.69,2708.60
1999,202002,20200229,2726068000,고산2동,2599.19,2085.08,1549.93,1509.08,1406.71,1800.18,...,11057.90,11587.25,12424.47,11691.56,10228.67,7664.57,6384.05,5682.11,4365.86,3399.61


### 서울시는 27일 “코로나19 장기화에 따라 지속가능한 안전 및 방역체계 확보를 위해 오는 4월 1일 수요일부터 24시까지만 열차를 운행한다”고 발표했다. 이에 따라 새벽 5시 30분부터 운행을 시작해 다음 날 새벽 1시까지 운행하던 모든 열차는 자정까지 한 시간씩 단축 운행한다.2020. 3. 27.

### skt 05를 절반 한것을 06시 이전으로 생각하면 되겠네. 1시는 24시 이후로.

## 선택해야하는게... 06-07이런걸 06에 맞출건지 07에 맞출건지. skt는 그 순간을 잰것일수도, 그 시간까지를 잰것일수도 있어서...
## 일단 06-07은 07로 생각하고 짠다.

In [19]:
time_col_need = ['STD_YMD', 'HDONG_NM', 'TMST_06', 'TMST_07', 'TMST_08', 'TMST_09', 'TMST_10', 'TMST_11', 'TMST_12', 'TMST_13', 'TMST_14', 'TMST_15', 'TMST_16', 'TMST_17', 'TMST_18', 'TMST_19', 'TMST_20','TMST_21','TMST_22','TMST_23', 'TMST_00', 'TMST_01']
time2002need = time2002[time_col_need]
time2002need['TMST_06'] = (time2002need['TMST_06'])/2
time_bt_col = ['사용일자', '동', '06시 이전', '06 ~ 07', '07 ~ 08', '08 ~ 09', '09 ~ 10', '10 ~ 11', '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15', '15 ~ 16', '16 ~ 17', '17 ~ 18', '18 ~ 19', '19 ~ 20', '20 ~ 21', '21 ~ 22', '22 ~ 23', '23 ~ 24', '24시 이후']
time2002need.columns = time_bt_col
time2002need

<ipython-input-19-ffc41c27200d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time2002need['TMST_06'] = (time2002need['TMST_06'])/2


,사용일자,동,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
0,20200201,소공동,1523.625,3418.62,5281.12,6511.74,6638.14,7583.97,8100.92,7558.40,...,7322.32,8070.40,6488.70,4890.46,4304.38,3683.57,4002.98,2894.37,1393.34,1144.91
1,20200201,회현동,1951.100,5185.70,8461.13,10632.62,10605.26,12262.11,12911.16,12305.87,...,11364.08,10725.66,8140.20,6399.03,5566.32,5533.92,4715.67,3334.92,1586.96,1170.81
2,20200201,명동,1801.805,4707.65,7198.98,9182.75,9864.41,10970.79,11471.71,11221.18,...,10587.30,10697.87,8781.11,6420.97,5091.03,5025.40,4415.18,3415.71,1534.52,1139.65
3,20200201,필동,1028.305,2456.02,4065.44,4588.57,4622.24,5209.10,5028.23,5202.56,...,5010.95,5702.29,5089.43,3959.67,3151.50,3048.29,2951.06,2017.30,1321.52,933.49
4,20200201,장충동,539.330,1327.15,2008.64,2460.44,3142.88,3703.56,3382.71,3156.54,...,2882.15,3099.08,2584.95,1811.88,1669.96,1664.65,1164.56,976.00,629.17,561.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,20200229,범물1동,292.125,680.62,935.62,1153.40,1385.50,1600.74,1610.45,1717.12,...,2093.72,2040.10,1849.73,1265.70,1041.32,905.75,712.61,628.77,485.96,388.68
1997,20200229,범물2동,434.835,913.40,973.74,1093.58,1210.56,1274.88,1283.35,1293.56,...,1296.90,1351.59,1351.30,1246.78,1140.83,1108.22,1031.76,1024.64,947.07,896.23
1998,20200229,고산1동,849.500,2362.88,3262.55,4085.70,4711.77,5121.46,5793.20,6019.40,...,6595.47,6772.06,6391.21,5232.65,4209.28,3874.91,3198.69,2708.60,2162.60,1758.41
1999,20200229,고산2동,1564.940,4528.29,5827.22,6832.27,8173.87,8789.60,9333.67,10107.46,...,12424.47,11691.56,10228.67,7664.57,6384.05,5682.11,4365.86,3399.61,2599.19,2085.08


In [20]:
def time_prepro(time_df):#, sta_dong_df):
    time_col_need = ['STD_YMD', 'HDONG_NM', 'TMST_06', 'TMST_07', 'TMST_08', 'TMST_09', 'TMST_10', 'TMST_11', 'TMST_12', 'TMST_13', 'TMST_14', 'TMST_15', 'TMST_16', 'TMST_17', 'TMST_18', 'TMST_19', 'TMST_20','TMST_21','TMST_22','TMST_23', 'TMST_00', 'TMST_01']
    time_df = time_df[time_col_need]
    time_df['TMST_06'] = (time_df['TMST_06'])/2#5시 반부터 운행한다니까...
    time_bt_col = ['사용일자', '동', '06시 이전', '06 ~ 07', '07 ~ 08', '08 ~ 09', '09 ~ 10', '10 ~ 11', '11 ~ 12', '12 ~ 13', '13 ~ 14', '14 ~ 15', '15 ~ 16', '16 ~ 17', '17 ~ 18', '18 ~ 19', '19 ~ 20', '20 ~ 21', '21 ~ 22', '22 ~ 23', '23 ~ 24', '24시 이후']
    time_df.columns = time_bt_col
    return time_df

In [21]:
# 일부열끼리만 더하고 싶으면 이거 쓰면 된다
# df['total'] = df[['drinking', 'drugs', 'both']].sum(axis=1) 

In [22]:
#시간 전처리 
time2002need = time_prepro(time2002)#, sta_dong)
time2003need = time_prepro(time2003)#, sta_dong)
time2004need = time_prepro(time2004)#, sta_dong)
time2005need = time_prepro(time2005)#, sta_dong)

<ipython-input-20-a758ef7e4bcd>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  time_df['TMST_06'] = (time_df['TMST_06'])/2#5시 반부터 운행한다니까...


In [23]:
time_need = pd.concat([time2002need, time2003need, time2004need, time2005need])
time_need

,사용일자,동,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
0,20200201,소공동,1523.625,3418.62,5281.12,6511.74,6638.14,7583.97,8100.92,7558.40,...,7322.32,8070.40,6488.70,4890.46,4304.38,3683.57,4002.98,2894.37,1393.34,1144.91
1,20200201,회현동,1951.100,5185.70,8461.13,10632.62,10605.26,12262.11,12911.16,12305.87,...,11364.08,10725.66,8140.20,6399.03,5566.32,5533.92,4715.67,3334.92,1586.96,1170.81
2,20200201,명동,1801.805,4707.65,7198.98,9182.75,9864.41,10970.79,11471.71,11221.18,...,10587.30,10697.87,8781.11,6420.97,5091.03,5025.40,4415.18,3415.71,1534.52,1139.65
3,20200201,필동,1028.305,2456.02,4065.44,4588.57,4622.24,5209.10,5028.23,5202.56,...,5010.95,5702.29,5089.43,3959.67,3151.50,3048.29,2951.06,2017.30,1321.52,933.49
4,20200201,장충동,539.330,1327.15,2008.64,2460.44,3142.88,3703.56,3382.71,3156.54,...,2882.15,3099.08,2584.95,1811.88,1669.96,1664.65,1164.56,976.00,629.17,561.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2134,20200531,범물1동,243.450,601.41,808.78,1243.35,1618.70,1797.39,1938.33,2251.36,...,2519.71,2341.32,2344.67,2002.38,1815.54,1398.37,996.67,592.60,598.19,402.80
2135,20200531,범물2동,361.035,851.21,880.83,981.90,1116.31,1085.08,1165.23,1191.27,...,1273.39,1262.05,1276.90,1255.39,1253.99,1139.82,1035.93,903.40,860.02,797.33
2136,20200531,고산1동,1090.140,3265.47,3656.56,5133.17,6404.87,6868.41,7944.86,8371.79,...,8484.01,8634.74,8781.90,8150.37,7550.56,6208.88,5092.64,3983.79,2934.21,2235.49
2137,20200531,고산2동,1488.200,4006.03,5622.52,9543.30,11590.12,13358.48,14187.36,15645.27,...,17122.82,16311.76,14701.17,13293.80,11671.12,9868.01,7015.46,4513.15,3332.81,2218.75


# 일단 동-전철 매칭이 애매하니까 수범형 했다던 노원역 해보자

In [52]:
def bt_station(bus_df, station):
    bt = bus_df[bus_df['지하철역']==station]
    return bt

In [53]:
def time_dong(time_df, dong):
    time = time_df[time_df['동']==dong]
    return time

## 노원역, 상계2동 corr

In [54]:
bt_downnw = bt_station(busT_down_need, '노원')
bt_downnw

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
39,20200201,노원,119.773,655.679,829.196,1962.43,3410.45,2711.78,2972.82,3752.88,...,4116.81,4213.55,5137.95,5048.89,3795.88,2819.27,2774.74,2363.21,1948.61,299.432
264,20200202,노원,228.891,955.304,567.46,1093.59,2072.74,1896.3,1947.17,2630.66,...,3409.53,3533.51,3582.79,3754.46,2608.41,2063.2,1940.81,1500.51,1287.51,195.511
489,20200203,노원,147.173,850.841,1684.82,5000.8,5270.62,3522.94,2872.93,3121.28,...,4103.97,4057.98,5528.17,8775.16,8361.24,4108.57,3059.96,2554.06,1545.31,745.061
714,20200204,노원,180.508,789.914,1667.77,4969.36,4688.57,3270.74,2922.07,3114.92,...,3897.12,4225.73,5276.38,8471.52,8272.5,4256.59,3022.35,2812.53,1845.19,742.087
939,20200205,노원,141.794,835.353,1607.52,4646.85,4867.24,3008.5,2630.9,2746.49,...,3745.22,4075.04,5190.9,8704.94,8429.05,4055.01,3171.88,2809.68,1809.42,755.209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,20200527,노원,164.814,919.569,2125.64,4622.49,3994.04,2817.24,2410.59,2558.46,...,3505.76,4131.13,4976.76,8721.27,7265.67,3627.44,3055.99,2613.92,1966.98,348.111
6032,20200528,노원,188.661,870.274,2139.17,4608.49,3981.65,2836,2508.88,2589.52,...,3389.81,3668.23,5224.68,8010.47,7124.99,3604.33,3056.61,2413.03,2053.97,336.242
6254,20200529,노원,137.316,843.514,2064.27,4579.72,4104.4,2799.14,2553.18,2636.17,...,3781.48,4125.52,5442.86,8632.82,7962.84,4205.5,3233.72,2729.73,2320.8,398.368
6476,20200530,노원,144.704,641.932,828.2,2102.83,3048.02,2496.92,2809.42,3348.21,...,3697.65,3696.11,4179.49,4011.69,3485.21,2664.71,2663.17,2082.82,1713.36,266.317


In [55]:
trainnw = bt_station(group_down, '노원')
trainnw

,사용일자,지하철역,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
7548,20200201,노원,603,695,1012,1552,1554,1613,1839,2075,...,2333,2630,2589,2240,1827,1958,1804,1733,853,8
7790,20200202,노원,427,498,674,1231,1206,1179,1054,1437,...,1860,1773,1830,1774,1657,1774,1494,1179,622,6
8033,20200203,노원,805,1710,4274,4100,2255,1981,2119,2067,...,2388,2457,2894,3262,2103,2067,1872,1888,849,231
8275,20200204,노원,743,1654,4239,4201,2298,1817,1837,1974,...,2353,2495,2734,3305,2051,2020,1825,1963,940,251
8517,20200205,노원,724,1582,4167,4156,2247,1674,1731,1840,...,2169,2403,2642,3218,2017,2045,1832,1946,934,225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35624,20200527,노원,751,1627,3950,3730,2135,1738,1708,1882,...,2076,2330,2607,2917,1852,1820,1650,1844,568,8
35866,20200528,노원,780,1632,3903,3834,2146,1754,1661,1784,...,2088,2264,2497,2963,1769,1775,1495,1840,553,5
36108,20200529,노원,736,1550,3829,3663,2165,1721,1704,1747,...,2080,2438,2556,3108,2061,1959,1674,2074,826,6
36350,20200530,노원,555,668,1019,1456,1626,1673,1727,2077,...,2165,2159,2271,1952,1489,1721,1502,1366,616,2


In [56]:
timenw = time_dong(time_need, '상계2동')
timenw

,사용일자,동,06시 이전,06 ~ 07,07 ~ 08,08 ~ 09,09 ~ 10,10 ~ 11,11 ~ 12,12 ~ 13,...,15 ~ 16,16 ~ 17,17 ~ 18,18 ~ 19,19 ~ 20,20 ~ 21,21 ~ 22,22 ~ 23,23 ~ 24,24시 이후
27,20200201,상계2동,620.135,1491.39,3022.03,4509.41,4417.77,4882.08,5440.38,5660.67,...,5831.10,6425.63,6446.96,5031.13,4050.48,3549.36,3076.18,2479.60,1287.65,1108.00
96,20200202,상계2동,533.925,1279.27,2151.93,3508.22,4051.80,3906.08,4218.20,4741.73,...,4924.26,5060.97,4683.16,4021.47,3233.37,2707.92,2140.05,1589.70,1338.58,859.58
165,20200203,상계2동,907.540,3796.31,5648.06,5117.31,3979.61,3838.92,3935.90,4332.57,...,4910.75,5858.20,7465.18,7311.64,4310.33,3268.17,2872.23,2043.73,852.18,565.59
234,20200204,상계2동,842.585,3456.26,5676.15,4956.77,4194.46,3954.05,4108.41,4398.82,...,5102.83,5709.10,7956.69,7151.17,4189.71,3543.04,3114.96,2098.40,1358.26,745.68
303,20200205,상계2동,816.885,3294.98,5709.45,5149.26,4115.46,3720.61,4062.13,4146.54,...,4857.44,5841.87,7712.98,7048.05,4180.61,3517.25,3161.61,2271.59,1274.93,766.69
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,20200527,상계2동,986.960,4076.45,5998.80,4860.13,4150.72,4011.35,4106.23,4051.77,...,5334.99,6290.33,7692.50,7169.87,4550.10,3891.17,3802.05,2448.56,1193.74,776.86
1890,20200528,상계2동,975.780,3800.13,5798.75,4921.78,3960.03,3763.27,4018.25,4199.67,...,5174.63,6263.91,7641.71,7173.95,4606.08,3710.34,3531.71,2418.54,1231.11,794.20
1959,20200529,상계2동,931.265,4138.99,5626.96,4713.91,4108.96,4016.20,4081.26,4240.66,...,5278.95,6351.22,7763.04,7615.94,5215.56,4307.70,3770.12,2804.83,1278.18,801.45
2028,20200530,상계2동,674.895,2150.87,3465.26,4657.11,4963.47,5085.15,5775.72,6048.54,...,5895.51,6406.07,6286.03,5361.50,4424.52,4127.43,3772.83,2636.55,1703.14,1010.30


In [57]:
def corr(df1, df2):
    cor_list = []
    for i in range(len(df1)):
        d1 = pd.DataFrame(list(df1.iloc[i][2:]))
        d2 = pd.DataFrame(list(df2.iloc[i][2:]))
        cor = pd.concat([d1, d2], axis=1)
        cor_list.append(float(cor.corr().iloc[0][1:]))
    return sum(cor_list)/len(cor_list)

In [58]:
corr(timenw, bt_downnw)

0.754103654435394

In [59]:
corr(timenw, trainnw)

0.789453508664938

## train이 corr 조금 더 높게 나온다. 시간있으면 다른 역도 해보기.